In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# !mv /content/drive/MyDrive/400-620 /content/dataset

Mounted at /content/drive


In [ ]:
!git clone https://github.com/pjreddie/darknet
%cd darknet
#line 293- src/image.c add - printf("{\"class_id\": %d, \"name\": \"%s\", \"x\": %f, \"y\": %f, \"width\": %f, \"height\": %f}\n",
       #class, names[class], b.x, b.y, b.w, b.h);

Cloning into 'darknet'...
remote: Enumerating objects: 5955, done.
remote: Total 5955 (delta 0), reused 0 (delta 0), pack-reused 5955
Receiving objects: 100% (5955/5955), 6.37 MiB | 12.65 MiB/s, done.
Resolving deltas: 100% (3932/3932), done.
/content/darknet


In [ ]:
!make
!wget https://pjreddie.com/media/files/yolov3.weights

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/utils.c -o obj/utils.o
In file included from /usr/include/string.h:495,
                 from ./src/utils.c:3:
In function ‘strncpy’,
    inlined from ‘copy_string’ at ./src/utils.c:426:5:
/usr/include/x86_64-linux-gnu/bits/string_fortified.h:106:10: warning: ‘__builtin_strncpy’ specified bound depends on the length of the source argument [-Wstringop-overflow=]
  106 |   return __builtin___strncpy_chk (__dest, __src, __len, __bos (__dest));
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
./src/utils.c: In function ‘copy_string’:
./src/utils.c:426:22: note: length computed here
  426 |     strncpy(copy, s, strlen(s)+1);
      |                      ^~~~~~~~~
gcc -Iinclude/ -Isrc/ -Wa

In [ ]:
import os
import re
import subprocess
import json

dataset_path = '/content/drive/MyDrive/scene_11'
bboxes_path = '/content/darknet/bboxes_new'
if not os.path.exists(bboxes_path):
    os.makedirs(bboxes_path)

for filename in sorted(os.listdir(dataset_path)):

    image_number = int(re.findall(r'\d+', filename)[0])
    if image_number < 200 or image_number >= 275:
        continue
    padded_image_number = str(image_number).zfill(3)  # Pad image number with zeros
    file_path = os.path.join(dataset_path, filename)
    command = f'./darknet detect cfg/yolov3.cfg yolov3.weights {file_path}'
    output = subprocess.check_output(command, shell=True, text=True)

    bounding_boxes = []
    for line in output.splitlines():
        if line.startswith('{'):
            bounding_box = json.loads(line)
            if not bounding_box['name'] == 'car' or bounding_box['name'] == 'truck':
                bounding_boxes.append(bounding_box)

    for box_idx, box in enumerate(bounding_boxes):

        box['x'] = int(box['x'] * 512)
        box['y'] = int(box['y'] * 256)
        box['width'] = int(box['width'] * 512)
        box['height'] = int(box['height'] * 256)
    
    # Use padded_image_number when saving the JSON file
    with open(os.path.join(bboxes_path, f'bounding_boxes{padded_image_number}.json'), 'w') as f:
        json.dump(bounding_boxes, f)

    
# command = './darknet detect cfg/yolov3.cfg yolov3.weights data/12.jpeg'
# output = subprocess.check_output(command, shell=True, text=True)

In [ ]:
%cd /content/bboxes
!zip bbox.zip *

/content/bboxes
  adding: bounding_boxes000.json (stored 0%)
  adding: bounding_boxes001.json (deflated 47%)
  adding: bounding_boxes002.json (stored 0%)
  adding: bounding_boxes003.json (stored 0%)
  adding: bounding_boxes004.json (stored 0%)
  adding: bounding_boxes005.json (deflated 15%)
  adding: bounding_boxes006.json (deflated 59%)
  adding: bounding_boxes007.json (deflated 47%)
  adding: bounding_boxes008.json (deflated 64%)
  adding: bounding_boxes009.json (deflated 49%)
  adding: bounding_boxes010.json (deflated 47%)
  adding: bounding_boxes011.json (deflated 49%)
  adding: bounding_boxes012.json (deflated 61%)
  adding: bounding_boxes013.json (deflated 62%)
  adding: bounding_boxes014.json (deflated 60%)
  adding: bounding_boxes015.json (deflated 60%)
  adding: bounding_boxes016.json (deflated 59%)
  adding: bounding_boxes017.json (deflated 61%)
  adding: bounding_boxes018.json (deflated 48%)
  adding: bounding_boxes019.json (deflated 59%)
  adding: bounding_boxes020.json (de

In [ ]:
os.makedirs('/content/dataset')

In [ ]:
%cd /content/bboxes/bboxes_new
!mv * /content/bboxes

/content/bboxes/bboxes_new


# Midas

In [ ]:
!pip install timm
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117cu102/torch-1.5.1-cp36-cp36m-win_amd64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.8 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu117cu102/torch-1.5.1-cp36-cp36m-win_amd64.whl, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
%cd /content/
img_folder = '/content/midas_images'
bbox_folder = '/content/assets_without_depth'

try:
    os.makedirs(img_folder)
except:
    pass

try:
    os.makedirs(bbox_folder)
except:
    pass

/content


In [ ]:
# add all bounding box jsons into '/content/assets_without_depth' path
# add all frames for MiDAS into '/content/midas_images'

In [ ]:
import re
import cv2
import torch
import urllib.request
import numpy as np

import matplotlib.pyplot as plt

model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_large_384.pt" to /root/.cache/torch/hub/checkpoints/dpt_large_384.pt
100%|██████████| 1.28G/1.28G [00:49<00:00, 28.1MB/s]


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

DPTDepthModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (act): GELU(approximate='none')
            (d

In [ ]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


In [ ]:
img_folder = '/content/drive/MyDrive/scene_11'
midas_results = '/content/midas_outputs'
if not os.path.exists(midas_results):
    os.makedirs(midas_results)

for i, image_name in enumerate(sorted(os.listdir(img_folder))):

    image_number = int(re.findall(r'\d+', image_name)[0])
    padded_image_number = str(image_number).zfill(3)  # Pad image number with zeros
    img_path = os.path.join(img_folder, image_name)

    img = cv2.imread(img_path)
    img = cv2.resize(img, (512, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    output = prediction.cpu().numpy()
    # output = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    np.save(f'/content/midas_outputs/frame_midas{padded_image_number}.npy', output)

#MiDAS

In [ ]:
import json
import numpy as np

vehicles_without_depth_path = '/content/vehicles_without_depth'
midas_path = '/content/midas_outputs'

vehicles_with_depth_path = '/content/vehicles_with_depth'
if not os.path.exists(vehicles_with_depth_path):
    os.makedirs(vehicles_with_depth_path)

for i, file_name in enumerate(sorted(os.listdir(vehicles_without_depth_path))):

    if not file_name.endswith('.json'):
        continue 
    file_number = int(re.findall(r'\d+', file_name)[0])
    padded_file_number = str(file_number).zfill(3)  # Pad image number with zeros

    midas_data = np.load(f'/content/midas_outputs/frame_midas{padded_file_number}.npy')
    file_object = open(os.path.join(vehicles_without_depth_path, file_name))
    asset_json_data = json.load(file_object)

    for idx in range(len(asset_json_data)):
        
        if bool(asset_json_data[idx]):

            asset_json_data[idx]['z'] = midas_data[int(asset_json_data[idx]['v']), int(asset_json_data[idx]['u'])]
  

    json_object = json.dumps(str(asset_json_data))

    # # Writing to sample.json
    with open(os.path.join(vehicles_with_depth_path, f'vehicles_with_depth{padded_file_number}.json'), 'w') as outfile:
        outfile.write(json_object)

In [ ]:
%cd /content/vehicles_with_depth
!zip vehicles_with_depth.zip *

/content/vehicles_with_depth
  adding: vehicles_with_depth000.json (deflated 41%)
  adding: vehicles_with_depth001.json (deflated 58%)
  adding: vehicles_with_depth002.json (deflated 45%)
  adding: vehicles_with_depth003.json (deflated 50%)
  adding: vehicles_with_depth004.json (deflated 58%)
  adding: vehicles_with_depth005.json (deflated 56%)
  adding: vehicles_with_depth006.json (deflated 47%)
  adding: vehicles_with_depth007.json (deflated 59%)
  adding: vehicles_with_depth008.json (deflated 49%)
  adding: vehicles_with_depth009.json (deflated 55%)
  adding: vehicles_with_depth010.json (deflated 56%)
  adding: vehicles_with_depth011.json (deflated 51%)
  adding: vehicles_with_depth012.json (deflated 52%)
  adding: vehicles_with_depth013.json (deflated 54%)
  adding: vehicles_with_depth014.json (deflated 59%)
  adding: vehicles_with_depth015.json (deflated 53%)
  adding: vehicles_with_depth016.json (deflated 59%)
  adding: vehicles_with_depth017.json (deflated 58%)
  adding: vehicle